In [1]:
import pybaseball
from pybaseball import statcast
from pybaseball import  statcast_pitcher



from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.tree import plot_tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer

from scipy.stats import beta, norm, binom
import scipy.optimize
from scipy.special import betaln

import matplotlib.pyplot as plt

import seaborn as sns

import pandas as pd

import numpy as np

import itertools

import xgboost as xgb

import joblib

import wandb

import os

from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append(os.path.abspath(".."))

In [2]:
def load_production_model(
    entity, project, model_name="pitch-odds-model"
):
    """
    Load production model pipeline from W&B artifact.
    Returns: model
    """

    # Login to W&B (uses WANDB_API_KEY env variable)
    wandb.login(key=os.environ["WANDB_API_KEY"])

    api = wandb.Api()

    # Fetch the production artifact
    artifact = api.artifact(
        f"{entity}/{project}/{model_name}:production", type="model"
    )

    # Download the artifact locally
    artifact_path = artifact.download()

    # Load Keras model
    model_file = f"{artifact_path}/model.pkl"
    model = joblib.load(model_file)

    return model

In [3]:
# Try and load model pipeline
try:
    ENTITY = 'chris-r-thompson1212-university-of-denver'
    PROJECT = "money-ball"
    model = load_production_model(
        ENTITY,
        PROJECT,
    )
    print("Model Loaded Successfully")
except FileNotFoundError:
    print("Error: unable to load model pipeline")
    model = None
    tokenizer = None
    maxlen = None

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/christopherthompson/.netrc
wandb: Currently logged in as: chris-r-thompson1212 (chris-r-thompson1212-university-of-denver) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb:   1 of 1 files downloaded.  


Model Loaded Successfully


In [4]:
pybaseball.cache.enable()
df = statcast(start_dt='2023-06-24', end_dt='2023-06-25')

This is a large query, it may take a moment to complete


100%|██████████| 2/2 [00:00<00:00,  2.86it/s]


In [5]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
2513,SI,2023-06-25,102.5,-0.26,6.92,"Bautista, Félix",606192,642585,strikeout,foul_tip,...,1,0.54,0.59,0.59,65.5,<NA>,<NA>,<NA>,<NA>,<NA>
2628,FS,2023-06-25,91.0,-0.53,6.79,"Bautista, Félix",606192,642585,None,foul,...,1,2.21,-0.06,-0.06,57.5,<NA>,<NA>,<NA>,<NA>,<NA>
2724,SI,2023-06-25,101.7,-0.34,6.84,"Bautista, Félix",606192,642585,None,foul,...,1,0.37,0.51,0.51,62.4,<NA>,<NA>,<NA>,<NA>,<NA>
2812,SI,2023-06-25,101.2,-0.14,6.99,"Bautista, Félix",606192,642585,None,swinging_strike,...,1,0.62,0.45,0.45,63.5,<NA>,<NA>,<NA>,<NA>,<NA>
2926,SI,2023-06-25,103.4,-0.34,6.9,"Bautista, Félix",606192,642585,None,ball,...,1,0.42,0.4,0.4,59.0,<NA>,<NA>,<NA>,<NA>,<NA>


In [6]:
preds = model.predict_proba(df.iloc[[0]])

In [7]:
print(preds)

[[0.3391406  0.34920987 0.12256458 0.18908495]]


In [8]:
user_input = {
    "pitcher": 642585,
    "batter": 606192,
    "balls": 3,
    "strikes": 2,
    "game_year": 2023,
    "outs_when_up": 1,
    "inning": 9,
    "at_bat_number": 23,
    "bat_score": 3,
    "fld_score": 2,
    "game_type": "R",
    "stand": "R",
    "p_throws": "L",
    "if_fielding_alignment": "Standard",
    "of_fielding_alignment": "Standard",
}

user_df = pd.DataFrame([user_input])

In [9]:
preds = model.predict_proba(user_df)

In [10]:
print(preds)

[[0.30115962 0.23911521 0.18439466 0.2753305 ]]


In [12]:
heldout_df = statcast(start_dt="2025-05-25", end_dt="2025-06-25")
# heldout_df.to_parquet("heldout_games.parquet")

heldout_df.to_parquet(
    "s3://statcast-mlb-raw/pitches/heldout_games.parquet",
    index=False
)

This is a large query, it may take a moment to complete


100%|██████████| 32/32 [00:17<00:00,  1.85it/s]
